In [1]:
from utils.ArticlesHandler import ArticlesHandler
from utils import solve, embedding_matrix_2_kNN, get_rate, accuracy, precision, recall, f1_score
from utils import Config
import time
import numpy as np
import scipy.sparse as sp
from postprocessing.SelectLabelsPostprocessor import SelectLabelsPostprocessor
from pygcn.utils import encode_onehot, accuracy, load_from_features
from model import AGNN
import torch
import torch.nn.functional as F
import torch.optim as optim

Import config file and check some values

In [2]:
config = Config(file='config')

assert (config.num_fake_articles + config.num_real_articles > 
        config.num_nearest_neighbours), "Can't have more neighbours than nodes!"

print("Method of decomposition:", config.method_decomposition_embedding)

Method of decomposition: parafac


Import the articles and decompose the tensor.

In [3]:
print("Loading dataset", config.dataset_name)
articles = ArticlesHandler(config)

print("Performing decomposition...")
C = articles.get_tensor()

Loading dataset Random Poltical News Dataset
Performing decomposition...


/media/benamira/19793564030D4273/MCsBackup/3A/OMA/Projet/misinformation-detection-tensor-embeddings/tensorly/decomposition/_tucker.py:60: Warning: Given only one int for 'rank' intead of a list of 3 modes. Using this rank for all modes.
  warnings.warn(message, Warning)


Get the labels

In [4]:
config.set("num_unknown_labels", 195)

In [5]:
labels = articles.articles.labels
all_labels = articles.articles.labels_untouched

In [6]:
index_higest_similarities = np.argpartition(C, 5, axis=1)[:, :5]
adj_mat = np.zeros((150, 150), dtype=int)
for i, neightbors in enumerate(index_higest_similarities):
    adj_mat[i, neightbors] = 1
# Force symetric
adj_mat = adj_mat + np.transpose(adj_mat)
adj_mat[adj_mat == 2] = 1

In [7]:
adj, features, all_labels = load_from_features(C, all_labels, config)
_, _, labels = load_from_features(C, labels, config)

In [8]:
# idx_train = range(150)
# idx_val = range(150, 175)
# idx_test = range(175, 200)
print(labels)
idx_train = np.where(labels)[0]
idx_val = np.where(1 - abs(labels))[0][:90]
idx_test = np.where(1 - abs(labels))[0][90:]

print(len(idx_train))

idx_train = torch.LongTensor(idx_train)
idx_val = torch.LongTensor(idx_val)
idx_test = torch.LongTensor(idx_test)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,
        2, 0, 0, 0, 2, 0, 2, 1, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2,
        1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 1,
        2, 1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 1, 2, 2, 1,
        1, 1, 2, 2, 2, 1])
75


In [9]:
import torch
import torch.nn as nn
from torch.nn.parameter import Parameter
from torch.autograd import Variable
import torch.nn.functional as F


class GraphAttentionLayer(nn.Module):

    def __init__(self, requires_grad=True):
        super(GraphAttentionLayer, self).__init__()
        if requires_grad:
            # unifrom initialization
            self.beta = Parameter(torch.Tensor(1).uniform_(
                0, 1), requires_grad=requires_grad)
        else:
            self.beta = Variable(torch.zeros(1), requires_grad=requires_grad)

    def forward(self, x, adj):


        # NaN grad bug fixed at pytorch 0.3. Release note:
        #     `when torch.norm returned 0.0, the gradient was NaN.
        #     We now use the subgradient at 0.0, so the gradient is 0.0.`
        norm2 = torch.norm(x, 2, 1).view(-1, 1)

        # add a minor constant (1e-7) to denominator to prevent division by
        # zero error
        if torch.cuda.is_available():
            cos = self.beta.cuda() * \
                  torch.div(torch.mm(x, x.t()), torch.mm(norm2, norm2.t()) + 1e-7)
        else:
            cos = self.beta * \
                  torch.div(torch.mm(x, x.t()), torch.mm(norm2, norm2.t()) + 1e-7)

        # neighborhood masking (inspired by this repo:
        # https://github.com/danielegrattarola/keras-gat)
        
        
        mask = (torch.ones(adj.shape) - adj) * -1e9
        masked = cos + mask

        # propagation matrix
        P = F.softmax(masked, dim=1)

        # attention-guided propagation
        output = torch.mm(P, x)
        return output

    def __repr__(self):
        return self.__class__.__name__ + ' (16 -> 16)'


class LinearLayer(nn.Module):

    def __init__(self, in_features, out_features, initializer=nn.init.xavier_uniform_):
        super(LinearLayer, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = Parameter(initializer(
            torch.Tensor(in_features, out_features)))

    def forward(self, input):
        # no bias
        return torch.mm(input, self.weight)

    def __repr__(self):
        return self.__class__.__name__ + ' (' \
            + str(self.in_features) + ' -> ' \
            + str(self.out_features) + ')'


class AGNN(nn.Module):

    def __init__(self, nfeat, nhid, nclass, nlayers, dropout_rate):
        super(AGNN, self).__init__()

        self.layers = nlayers
        self.dropout_rate = dropout_rate

        self.embeddinglayer = LinearLayer(nfeat, nhid)
        nn.init.xavier_uniform_(self.embeddinglayer.weight)

        self.attentionlayers = nn.ModuleList()
        # for Cora dataset, the first propagation layer is non-trainable
        # and beta is fixed at 0
        self.attentionlayers.append(GraphAttentionLayer(requires_grad=False).cuda())
        for i in range(1, self.layers):
            if torch.cuda.is_available():
                self.attentionlayers.append(GraphAttentionLayer().cuda())
            else:
                self.attentionlayers.append(GraphAttentionLayer())


        self.outputlayer = LinearLayer(nhid, nclass)
        nn.init.xavier_uniform_(self.outputlayer.weight)

    def forward(self, x, adj):
        x = F.relu(self.embeddinglayer(x))
        x = F.dropout(x, self.dropout_rate, training=self.training)

        for i in range(self.layers):
            x = self.attentionlayers[i](x, adj)

        x = self.outputlayer(x)
        x = F.dropout(x, self.dropout_rate, training=self.training)
        return F.log_softmax(x, dim=1)

In [12]:
cuda = False
hidden = 16
dropout = 0.5
lr = 0.01
weight_decay = 5e-4
fastmode = False
epochs = 450
layers =4

# Model and optimizer
model = AGNN(nfeat=features.shape[1],
                     nhid=hidden,
                     nclass=2,
                     nlayers=layers,
                     dropout_rate=0.5)

optimizer = optim.Adam(model.parameters(),
                       lr=lr, weight_decay=weight_decay)

if cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    all_labels = all_labels.cuda()
    idx_train = idx_train.cuda()
    idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()


def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    output = model(features, adj)

    
    loss_train = F.nll_loss(output[idx_train], all_labels[idx_train])
    acc_train = accuracy(output[idx_train], all_labels[idx_train])
    loss_train.backward()
    optimizer.step()

    if not fastmode:
        # Evaluate validation set performance separately,
        # deactivates dropout during validation run.
        model.eval()
        output = model(features, adj)

    loss_val = F.nll_loss(output[idx_val], all_labels[idx_val])
    acc_val = accuracy(output[idx_val], all_labels[idx_val])
    print('Epoch: {:04d}'.format(epoch+1),
          'loss_train: {:.4f}'.format(loss_train.item()),
          'acc_train: {:.4f}'.format(acc_train.item()),
          'loss_val: {:.4f}'.format(loss_val.item()),
          'acc_val: {:.4f}'.format(acc_val.item()),
          'time: {:.4f}s'.format(time.time() - t))


def test():
    model.eval()
    output = model(features, adj)
    loss_test = F.nll_loss(output[idx_test], all_labels[idx_test])
    acc_test = accuracy(output[idx_test], all_labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))


# Train model
t_total = time.time()
for epoch in range(epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# Testing
test()

Epoch: 0001 loss_train: 1.0692 acc_train: 0.4800 loss_val: 0.6570 acc_val: 0.5667 time: 0.0164s
Epoch: 0002 loss_train: 1.3695 acc_train: 0.4667 loss_val: 0.6532 acc_val: 0.6111 time: 0.0248s
Epoch: 0003 loss_train: 0.7126 acc_train: 0.5333 loss_val: 0.6539 acc_val: 0.6000 time: 0.0234s
Epoch: 0004 loss_train: 0.7185 acc_train: 0.4800 loss_val: 0.6649 acc_val: 0.6222 time: 0.0143s
Epoch: 0005 loss_train: 0.9660 acc_train: 0.4667 loss_val: 0.6922 acc_val: 0.5778 time: 0.0139s
Epoch: 0006 loss_train: 0.7320 acc_train: 0.5733 loss_val: 0.7363 acc_val: 0.5556 time: 0.0145s
Epoch: 0007 loss_train: 0.6301 acc_train: 0.6000 loss_val: 0.7851 acc_val: 0.5778 time: 0.0151s
Epoch: 0008 loss_train: 0.6461 acc_train: 0.6133 loss_val: 0.8319 acc_val: 0.6000 time: 0.0143s
Epoch: 0009 loss_train: 0.6490 acc_train: 0.5733 loss_val: 0.8760 acc_val: 0.6111 time: 0.0140s
Epoch: 0010 loss_train: 0.7857 acc_train: 0.4800 loss_val: 0.9166 acc_val: 0.6222 time: 0.0128s
Epoch: 0011 loss_train: 0.6688 acc_train

Epoch: 0098 loss_train: 0.5788 acc_train: 0.7467 loss_val: 1.1609 acc_val: 0.6444 time: 0.0157s
Epoch: 0099 loss_train: 0.4897 acc_train: 0.7600 loss_val: 1.1503 acc_val: 0.6444 time: 0.0168s
Epoch: 0100 loss_train: 0.5268 acc_train: 0.7200 loss_val: 1.1455 acc_val: 0.6444 time: 0.0171s
Epoch: 0101 loss_train: 0.4662 acc_train: 0.7467 loss_val: 1.1448 acc_val: 0.6444 time: 0.0169s
Epoch: 0102 loss_train: 0.5056 acc_train: 0.6933 loss_val: 1.1444 acc_val: 0.6444 time: 0.0130s
Epoch: 0103 loss_train: 0.5304 acc_train: 0.7067 loss_val: 1.1507 acc_val: 0.6444 time: 0.0129s
Epoch: 0104 loss_train: 0.5168 acc_train: 0.7067 loss_val: 1.1602 acc_val: 0.6444 time: 0.0130s
Epoch: 0105 loss_train: 0.5153 acc_train: 0.7067 loss_val: 1.1688 acc_val: 0.6556 time: 0.0128s
Epoch: 0106 loss_train: 0.5477 acc_train: 0.7067 loss_val: 1.1769 acc_val: 0.6667 time: 0.0143s
Epoch: 0107 loss_train: 0.5408 acc_train: 0.6533 loss_val: 1.1832 acc_val: 0.6444 time: 0.0141s
Epoch: 0108 loss_train: 0.5352 acc_train

Epoch: 0194 loss_train: 0.4537 acc_train: 0.7733 loss_val: 1.0597 acc_val: 0.7667 time: 0.0480s
Epoch: 0195 loss_train: 0.5095 acc_train: 0.7467 loss_val: 1.0597 acc_val: 0.7556 time: 0.0346s
Epoch: 0196 loss_train: 0.5280 acc_train: 0.7867 loss_val: 1.0567 acc_val: 0.7444 time: 0.0331s
Epoch: 0197 loss_train: 0.4485 acc_train: 0.7600 loss_val: 1.0534 acc_val: 0.7333 time: 0.0310s
Epoch: 0198 loss_train: 0.5096 acc_train: 0.7600 loss_val: 1.0448 acc_val: 0.7333 time: 0.0282s
Epoch: 0199 loss_train: 0.5132 acc_train: 0.6533 loss_val: 1.0341 acc_val: 0.7222 time: 0.0216s
Epoch: 0200 loss_train: 0.4585 acc_train: 0.7733 loss_val: 1.0228 acc_val: 0.7222 time: 0.0257s
Epoch: 0201 loss_train: 0.5455 acc_train: 0.7067 loss_val: 0.9991 acc_val: 0.6778 time: 0.0188s
Epoch: 0202 loss_train: 0.4822 acc_train: 0.7600 loss_val: 0.9810 acc_val: 0.6778 time: 0.0278s
Epoch: 0203 loss_train: 0.4729 acc_train: 0.7333 loss_val: 0.9672 acc_val: 0.6778 time: 0.0223s
Epoch: 0204 loss_train: 0.5152 acc_train

Epoch: 0283 loss_train: 0.4552 acc_train: 0.8133 loss_val: 1.0239 acc_val: 0.7333 time: 0.0159s
Epoch: 0284 loss_train: 0.4706 acc_train: 0.7333 loss_val: 1.0450 acc_val: 0.7444 time: 0.0182s
Epoch: 0285 loss_train: 0.4345 acc_train: 0.7600 loss_val: 1.0656 acc_val: 0.7556 time: 0.0114s
Epoch: 0286 loss_train: 0.5163 acc_train: 0.6933 loss_val: 1.0825 acc_val: 0.7556 time: 0.0178s
Epoch: 0287 loss_train: 0.5152 acc_train: 0.6933 loss_val: 1.0901 acc_val: 0.7556 time: 0.0137s
Epoch: 0288 loss_train: 0.4340 acc_train: 0.7733 loss_val: 1.0956 acc_val: 0.7556 time: 0.0172s
Epoch: 0289 loss_train: 0.4907 acc_train: 0.7067 loss_val: 1.1010 acc_val: 0.7556 time: 0.0171s
Epoch: 0290 loss_train: 0.4946 acc_train: 0.7600 loss_val: 1.1059 acc_val: 0.7556 time: 0.0208s
Epoch: 0291 loss_train: 0.5119 acc_train: 0.7733 loss_val: 1.1072 acc_val: 0.7556 time: 0.0186s
Epoch: 0292 loss_train: 0.5586 acc_train: 0.6667 loss_val: 1.1164 acc_val: 0.7556 time: 0.0226s
Epoch: 0293 loss_train: 0.4735 acc_train

Epoch: 0370 loss_train: 0.4309 acc_train: 0.7200 loss_val: 0.7508 acc_val: 0.7000 time: 0.0147s
Epoch: 0371 loss_train: 0.5548 acc_train: 0.6933 loss_val: 0.7724 acc_val: 0.7000 time: 0.0161s
Epoch: 0372 loss_train: 0.4668 acc_train: 0.7600 loss_val: 0.7898 acc_val: 0.7333 time: 0.0138s
Epoch: 0373 loss_train: 0.4306 acc_train: 0.8000 loss_val: 0.8052 acc_val: 0.7444 time: 0.0181s
Epoch: 0374 loss_train: 0.4385 acc_train: 0.7733 loss_val: 0.8198 acc_val: 0.7556 time: 0.0122s
Epoch: 0375 loss_train: 0.4633 acc_train: 0.6933 loss_val: 0.8352 acc_val: 0.7444 time: 0.0122s
Epoch: 0376 loss_train: 0.4598 acc_train: 0.7733 loss_val: 0.8468 acc_val: 0.7556 time: 0.0122s
Epoch: 0377 loss_train: 0.4471 acc_train: 0.8000 loss_val: 0.8482 acc_val: 0.7556 time: 0.0121s
Epoch: 0378 loss_train: 0.4225 acc_train: 0.8133 loss_val: 0.8439 acc_val: 0.7556 time: 0.0119s
Epoch: 0379 loss_train: 0.3983 acc_train: 0.8533 loss_val: 0.8315 acc_val: 0.7556 time: 0.0122s
Epoch: 0380 loss_train: 0.4591 acc_train